## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

### 1. Изучение общей информации. 

In [3]:
import pandas as pd
credit_data = pd.read_csv('/datasets/data.csv')
credit_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Для проведения анализа необходимо сначала обработать таблицу - заполнить пропуски, лемматизировать и категоризировать цель кредита, найти дубликаты, привести к нижему регистру столбец образование.

### 2. Предобработка данных

### Обработка пропусков

In [5]:
credit_data.isna().sum()


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
unique = credit_data['income_type'].unique()
print(unique)

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


In [7]:
for i in unique:
    type_of_employee = credit_data[credit_data['income_type'] == i]
    type_of_employee_median = type_of_employee['total_income'].median()
    credit_data.loc[credit_data['income_type'] == i, 'total_income'] = credit_data[credit_data['income_type'] == i]['total_income'].fillna(type_of_employee_median)
    

In [8]:
credit_data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

### Вывод

В нашей таблице пропуски значений в столбцах days_employed и total_income. Чтобы не терять значения, пропуски можно заполнить медианным значением, отфильтровав таблицу по каждой группе income_type. В столбце days_employed присутсвуют нереалистичные данные (100+ лет стажа), так как с другими данными они не коррелируются и нет других аномалий, эти данные не будут учитываться в анализе.

### Замена типа данных

In [9]:
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [10]:
credit_data['total_income'] = credit_data['total_income'].astype('int')
credit_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Вывод

Все типы данных соответствуют своим значениям.

### Обработка дубликатов

Для того, чтобы избежать дублей, необходимо привести столбец education к единому виду.

In [11]:
credit_data['education_lowercase'] = credit_data['education'].str.lower()
credit_data['education'] = credit_data['education_lowercase']
credit_data.tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,среднее
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,среднее
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,среднее
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,среднее
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047,на покупку автомобиля,среднее


In [12]:
credit_data.duplicated().sum()

71

In [13]:
credit_data.drop_duplicates().reset_index(drop = True)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,высшее
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,среднее
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,среднее
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,среднее
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,среднее
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,среднее
21450,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,среднее
21451,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,среднее
21452,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,среднее


### Вывод

Найдены и удалены 71 дублирующих строки.

### Лемматизация

In [14]:
from pymystem3 import Mystem
m = Mystem()

In [15]:
from pymystem3 import Mystem
m = Mystem()
def make_lemma(row):
    purpose_lemmatized = m.lemmatize(row)
    return purpose_lemmatized
    
credit_data['purpose_lemmatized'] = credit_data['purpose'].apply(make_lemma)
credit_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase,purpose_lemmatized
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,высшее,"[покупка, , жилье, \n]"
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,среднее,"[приобретение, , автомобиль, \n]"
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,среднее,"[покупка, , жилье, \n]"
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,среднее,"[дополнительный, , образование, \n]"
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,среднее,"[сыграть, , свадьба, \n]"
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,высшее,"[покупка, , жилье, \n]"
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,высшее,"[операция, , с, , жилье, \n]"
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,среднее,"[образование, \n]"
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,высшее,"[на, , проведение, , свадьба, \n]"
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,среднее,"[покупка, , жилье, , для, , семья, \n]"


### Вывод

После лемматизации получилось привести цели взятия кредита к единому виду для проведения дальнейшей категоризации.  


### Категоризация данных

Категоризация по цели получения кредита

In [16]:
lemmas = ' '.join(credit_data['purpose'])
lem_list = m.lemmatize(lemmas)

In [17]:
from collections import Counter
print(Counter(lem_list).most_common(20))

[(' ', 55201), ('недвижимость', 6367), ('покупка', 5912), ('жилье', 4473), ('автомобиль', 4315), ('образование', 4022), ('с', 2924), ('операция', 2610), ('свадьба', 2348), ('свой', 2235), ('на', 2233), ('строительство', 1881), ('высокий', 1375), ('получение', 1316), ('коммерческий', 1315), ('для', 1294), ('жилой', 1233), ('сделка', 944), ('дополнительный', 909), ('заниматься', 908)]


In [18]:

def categorized(row):
    try:
        if 'недвижимость' in row:
            return 'покупка недвижимости'
        if 'жилье' in row:
            return 'покупка недвижимости'
        if 'образование' in row:
            return 'образование'
        if 'автомобиль' in row:
            return 'покупка автомобиля'
        if  'машина' in row:
             return 'покупка автомобиля'
        if 'свадьба' in row:
            return 'свадьба'
    except: 
        return 'другая категория'
credit_data['loan_category'] = credit_data['purpose_lemmatized'].apply(categorized)

In [19]:
credit_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase,purpose_lemmatized,loan_category
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,высшее,"[покупка, , жилье, \n]",покупка недвижимости
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,среднее,"[приобретение, , автомобиль, \n]",покупка автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,среднее,"[покупка, , жилье, \n]",покупка недвижимости
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,среднее,"[дополнительный, , образование, \n]",образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,среднее,"[сыграть, , свадьба, \n]",свадьба


In [20]:
data_by_value = credit_data['loan_category'].value_counts().reset_index()
data_by_value.set_axis(['loan_category','total'],axis = 'columns',inplace = True)

print(data_by_value)

          loan_category  total
0  покупка недвижимости  10840
1    покупка автомобиля   4315
2           образование   4022
3               свадьба   2348


In [21]:
data_grouped = credit_data.groupby('loan_category')['debt'].sum().reset_index().sort_values('debt',ascending=False)
print(data_grouped)

          loan_category  debt
2  покупка недвижимости   782
1    покупка автомобиля   403
0           образование   370
3               свадьба   186


In [22]:
data_value = credit_data['loan_category'].value_counts().reset_index()
data_value.set_axis(['loan_category','total'],axis = 'columns',inplace = True)
print(data_value)

          loan_category  total
0  покупка недвижимости  10840
1    покупка автомобиля   4315
2           образование   4022
3               свадьба   2348


Категоризация по уровню дохода с помощью квантилей

In [23]:
credit_data['total_income'].quantile(q=[0.25,0.5,0.75])

0.25    107798.0
0.50    142594.0
0.75    195549.0
Name: total_income, dtype: float64

уровень дохода:
- низкий от 107798 до 142594
- средний от 142595 до 195549
- высокий от 195550

In [24]:
def income_level(income_total):
    if income_total < 107798:
        return 'очень низкий'
    if income_total < 142594:
        return 'низкий'
    if income_total <= 195549:
        return 'средний'
    return 'высокий'

In [25]:
credit_data['income_level'] = credit_data['total_income'].apply(income_level)
credit_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase,purpose_lemmatized,loan_category,income_level
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,высшее,"[покупка, , жилье, \n]",покупка недвижимости,высокий
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,среднее,"[приобретение, , автомобиль, \n]",покупка автомобиля,низкий
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,среднее,"[покупка, , жилье, \n]",покупка недвижимости,средний
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,среднее,"[дополнительный, , образование, \n]",образование,высокий
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,среднее,"[сыграть, , свадьба, \n]",свадьба,средний


In [26]:
data_by_income = credit_data['income_level'].value_counts().reset_index()
data_by_income.set_axis(['income_level','total'],axis = 'columns',inplace = True)
print(data_by_income)

   income_level  total
0       средний   6344
1  очень низкий   5381
2       высокий   5381
3        низкий   4419


In [27]:
data_grouped_by_income = credit_data.groupby('income_level')['debt'].sum().reset_index().sort_values('debt',ascending=False)
print(data_grouped)

          loan_category  debt
2  покупка недвижимости   782
1    покупка автомобиля   403
0           образование   370
3               свадьба   186


Категоризация по семейному положению

In [28]:
data_grouped_by_status = credit_data.groupby('family_status')['debt'].sum().reset_index().sort_values('debt',ascending=False)
print(data_grouped_by_status)

           family_status  debt
4        женат / замужем   931
3       гражданский брак   388
0  Не женат / не замужем   274
1              в разводе    85
2         вдовец / вдова    63


In [29]:
data_by_status = credit_data['family_status'].value_counts().reset_index()
data_by_status.set_axis(['family_status','total'],axis = 'columns',inplace = True)
print(data_by_status)

           family_status  total
0        женат / замужем  12380
1       гражданский брак   4177
2  Не женат / не замужем   2813
3              в разводе   1195
4         вдовец / вдова    960


Словарь для семейного статуса

In [30]:
family_status_dict = credit_data[['family_status','family_status_id']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict.sort_values('family_status_id', ascending = True)

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


### Вывод

Сделана категориризация по цели получения кредита, по уровню дохода и по семейному положению.

### 3. Исследование надежности заемщиков

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [31]:
credit_data.loc[credit_data['children'] == 20, 'children'] = 4
credit_data.loc[credit_data['children'] == -1, 'children'] = 5
print(credit_data['children'].value_counts())


                                    

0    14149
1     4818
2     2055
3      330
4      117
5       56
Name: children, dtype: int64


Значения '20' и '-1' в столбце дети являются артефактами, я их объеденила со значениями с близкими цифрами в полученном соотношении ratio.

In [32]:
data_final_ch = credit_data.groupby(['children']).agg({'debt':['sum', 'count']})
data_final_ch['ratio'] = data_final_ch['debt',   'sum'] / data_final_ch['debt', 'count']
print(data_final_ch.sort_values(by = 'ratio', ascending = False))

          debt            ratio
           sum  count          
children                       
4           12    117  0.102564
2          194   2055  0.094404
1          444   4818  0.092154
3           27    330  0.081818
0         1063  14149  0.075129
5            1     56  0.017857


### Вывод

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [33]:
data_pivot_fam = credit_data.pivot_table(index=['family_status'], values='debt', aggfunc=['sum', 'count'])
data_pivot_fam['ratio'] = data_pivot_fam['sum', 'debt'] / data_pivot_fam['count', 'debt']
data_pivot_fam.sort_values('ratio', ascending = False)

,sum,count,ratio
,debt,debt,
family_status,,,
Не женат / не замужем,274,2813,0.097405
гражданский брак,388,4177,0.092890
женат / замужем,931,12380,0.075202
в разводе,85,1195,0.071130
вдовец / вдова,63,960,0.065625


### Вывод

На долю людей,  кто находится или когда-либо был в браке приходится наименьшее число займов с просрочкой.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Выявление зависимости между уровнем дохода и возвратом кредита в срок с помощью сводной таблицы.

In [34]:
data_pivot_income = credit_data.pivot_table(index=['income_level'], values='debt', aggfunc=['sum', 'count'])
data_pivot_income['ratio'] = data_pivot_income['sum', 'debt'] / data_pivot_income['count', 'debt']
data_pivot_income.sort_values('ratio', ascending = False)

,sum,count,ratio
,debt,debt,
income_level,,,
низкий,385,4419,0.087124
средний,543,6344,0.085593
очень низкий,427,5381,0.079353
высокий,386,5381,0.071734


### Вывод

На долю людей со средним уровнем дохода приходится наименьшее число займов с просрочкой, чуть меньше на людей с низким уровнем дохода. У людей с высоким доходом самая высокая доля числа займов с просрочкой.

- Как разные цели кредита влияют на его возврат в срок?

Выявление зависимости как разные цели кредита влияют на его возврат в срок с помощью сводной таблицы.

In [35]:
data_pivot_cat = credit_data.pivot_table(index=['loan_category'], values='debt', aggfunc=['sum', 'count'])
data_pivot_cat['ratio'] = data_pivot_cat['sum', 'debt'] / data_pivot_cat['count', 'debt']
data_pivot_cat.sort_values('ratio', ascending = False)

,sum,count,ratio
,debt,debt,
loan_category,,,
покупка автомобиля,403,4315,0.093395
образование,370,4022,0.091994
свадьба,186,2348,0.079216
покупка недвижимости,782,10840,0.072140


Выявление зависимости как разные цели кредита и уровень дохода влияют на его возврат в срок с помощью сводной таблицы.

In [37]:
data_pivot_cat = credit_data.pivot_table(index=['loan_category'],columns = 'income_level', values='debt', aggfunc=['sum', 'count'])
data_pivot_cat['ratio_high'] = data_pivot_cat['sum', 'высокий'] / data_pivot_cat['count', 'высокий']
data_pivot_cat['ratio_low'] = data_pivot_cat['sum', 'низкий'] / data_pivot_cat['count', 'низкий']
data_pivot_cat['ratio_medium'] = data_pivot_cat['sum', 'средний'] / data_pivot_cat['count', 'средний']
data_pivot_cat['ratio_verylow'] = data_pivot_cat['sum', 'очень низкий'] / data_pivot_cat['count', 'очень низкий']
data_pivot_cat

sum                               count         \
income_level         высокий низкий очень низкий средний высокий низкий   
loan_category                                                             
образование               75     87           91     117     944    816   
покупка автомобиля        93     85          101     124    1095    942   
покупка недвижимости     187    172          181     242    2764   2176   
свадьба                   31     41           54      60     578    485   

                                          ratio_high ratio_low ratio_medium  \
income_level         очень низкий средний                                     
loan_category                                                                 
образование                  1046    1216   0.079449  0.106618     0.096217   
покупка автомобиля           1050    1228   0.084932  0.090234     0.100977   
покупка недвижимости         2681    3219   0.067656  0.079044     0.075179   
свадьба                       604     681   0.053633  0.084536     0.088106   

                     ratio_verylow  
income_level                        
loan_category                       
образование               0.086998  
покупка автомобиля        0.096190  
покупка недвижимости      0.067512  
свадьба                   0.089404

### Вывод

На долю людей, взявших кредит для покупки автомобиля, приходится наименьшее число займов с просрочкой, дальше в порядке убывания на образовние, свадьбу, покупку недвижимости. При этом почти одинаково возвращают кредиты люди с низким и средним уровнями дохода, люди с высоким уровнем дохода возвращают кредиты хуже вне зависимости от цели получения кредита.

###  4. Общий вывод

Самыми дисциплинированными группами заемщиков оказались люди со средним доходом, женатые и взявшие кредит на покупку автомобиля и образование. Аккуратными плательщиками являются так же люди, у которых есть от 1 до 4 детей. Без детей и с 5ю детьми на последних местах рейтинга.